# Production Scheduling Problem

In this example, we’ll be solving a scheduling problem. We have 2 offshore production plants in 2 locations and an estimated demand for our products.

We want to produce a schedule of production from both plants that meets our demand with the lowest cost.

A factory can be in 2 states:
* Off – Producing zero units
* On – Producing between its minimum and maximum production capacities

Both factories have `fixed costs`, that are incurred as long as the factory is on, and `variable costs`, a cost per unit of production. These vary month by month.

We also know that factory B is down for maintenance in month 5.

In [1]:
# importing libraries
import pandas as pd
import pulp as p

In [2]:
# Factory variables
factories = pd.read_csv('factory_variables.csv', index_col=['Month', 'Factory'])

In [3]:
factories

Max_Capacity  Min_Capacity  Variable_Cost  Fixed_Cost
Month Factory                                                       
1     A              100000         20000             10         500
      B               50000         20000              5         600
2     A              110000         20000             11         500
      B               55000         20000              4         600
3     A              120000         20000             12         500
      B               60000         20000              3         600
4     A              145000         20000              9         500
      B              100000         20000              5         600
5     A              160000         20000              8         500
      B                   0             0              0           0
6     A              140000         20000              8         500
      B               70000         20000              6         600
7     A              155000         20000              5         500
      B               60000         20000              4         600
8     A              200000         20000              7         500
      B              100000         20000              6         600
9     A              210000         20000              9         500
      B              100000         20000              6         600
10    A              197000         20000             10         500
      B              100000         20000             11         600
11    A               80000         20000              8         500
      B              120000         20000             10         600
12    A              150000         20000              8         500
      B              150000         20000             12         600

In [4]:
# Monthly Demand
demand = pd.read_csv('monthly_demand.csv', index_col='Month')

In [5]:
demand

,Demand
Month,
1,120000
2,100000
3,130000
4,130000
5,140000
6,130000
7,150000
8,170000
9,200000


As we have fixed costs and variable costs, we’ll need to model both production and the status of the factory i.e. whether it is on or off.

Production is modelled as an integer variable.

We have a value for production for each month for each factory, this is given by the tuples of our multi-index pandas DataFrame index.

In [10]:
production = p.LpVariable.dicts('production', ((month, factory) for month, factory in factories.index),
                               lowBound = 0,
                               cat = "Integer")

Factory status is modelled as a **binary variable**. It will have a value of 1 if the factory is on and a value of 0 when the factory is off.

Again this has a value for each month for each factory, again given by the index of our DataFrame.

In [11]:
factory_status = p.LpVariable.dicts('factory-status', ((month, factory) for month,factory in factories.index),
                                  cat='Binary')

## Defining Problem...

We instantiate our model and use LpMinimize as the aim is to minimise costs.

In [12]:
# LPP Problem
model = p.LpProblem('SCheduling-Problem', p.LpMinimize)

## Objective Function...
In our objective function we include our 2 costs:
* Our variable costs is the product of the variable costs per unit and production
* Our fixed costs is the factory status – 1 (on) or 0 (off) – multiplied by the fixed cost of production

In [13]:
# Defining Objective Function
model += p.lpSum([
                    [production[month,factory]*factories.loc[(month,factory), 'Variable_Cost'] for month,factory in factories.index]
                    + [factory_status[month,factory]*factories.loc[(month,factory), 'Fixed_Cost'] for month,factory in factories.index]
                ])

## Building up constraints...
Our monthly production should match monthly demand.

In [14]:
for month in demand.index:
    model += production[month,'A'] + production[month,'B'] == demand.loc[month,'Demand']

**An issue we run into here is that in linear programming we can’t use conditional constraints.**

For example we can’t add to our model that if the factory is off factory status must be 0, and if it is on factory status must be 1. Before we’ve solved our model though, we don’t know if the factory will be on or off in a given month.

In this case, construct constraints that have minimum and maximum capacities that are constant variables, which we multiply by the factory status.

Also, the monthly production should be less than monthly maximum capacity and greater than monthly minimum capacity or zero(in case factory is closed).

In [15]:
for month, factory in factories.index:
    model += production[month, factory] <= factories.loc[(month,factory),'Max_Capacity']*factory_status[month,factory]
    model += production[month, factory] >= factories.loc[(month,factory),'Min_Capacity']*factory_status[month,factory]

Also factory B was closed in month 5.

In [16]:
model += production[5,'B'] == 0
model += factory_status[5,'B'] == 0

# Solving model

In [17]:
status = p.LpStatus[model.solve()]
print(status)

Optimal


In [26]:
# result dataframe
result = []
for month,factory in factories.index:
    result.append([month, factory, factory_status[month,factory].varValue, int(production[month,factory].varValue)])
    
df = pd.DataFrame(result, columns=['Month', 'Factory', 'Status', 'Production'])

In [28]:
df.set_index(['Month', 'Factory'], inplace=True)
df

Status  Production
Month Factory                    
1     A           1.0       70000
      B           1.0       50000
2     A           1.0       45000
      B           1.0       55000
3     A           1.0       70000
      B           1.0       60000
4     A           1.0       30000
      B           1.0      100000
5     A           1.0      140000
      B           0.0           0
6     A           1.0       60000
      B           1.0       70000
7     A           1.0       90000
      B           1.0       60000
8     A           1.0       70000
      B           1.0      100000
9     A           1.0      100000
      B           1.0      100000
10    A           1.0      190000
      B           0.0           0
11    A           1.0       80000
      B           1.0       60000
12    A           1.0      100000
      B           0.0           0

In [30]:
# The minimized cost of production is given by
print(f' Cost = {p.value(model.objective)}')

 Cost = 12706400.0
